## データについて
各カラムの意味は以下の通りです.
<table>
  <thead>
  <tr>
    <td>.csvファイル</td>
    <td>学習モデル</td>
    <td>CVスコア</td>
    <td>暫定スコア</td>
    <td>備考</td>     
  </tr>
  </thead>
  <tr>
    <td>submission06102200.csv</td>
    <td>LR only</td> 
    <td>0.87363</td>
    <td>0.86669</td>
    <td>officer_idは100000のみ</td>
  </tr>
  <tr>
    <td>submission06111044.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.87809</td>
    <td>0.?????</td>
    <td>同上</td>
  </tr>
  <tr>
    <td>submission06111100.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.88333</td>
    <td>0.?????</td>
    <td>stop_hourを修正</td>
  </tr> 
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)</td> 
    <td>0.88728</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4,n_est=200)</td> 
    <td>0.88848</td>
    <td>0.?????</td>
    <td></td>
  </tr>   
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)とLRを10:1</td> 
    <td>0.88766</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4)</td> 
    <td>0.86897</td>
    <td>0.?????</td>
    <td>officer_idを追加した</td>
  </tr>  
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB(確か4と400)</td> 
    <td>0.?????</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>   
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB&LR&RF</td> 
    <td>0.88907</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>     
</table>


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from datetime import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#m = xgb.XGBRegressor()
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df = pd.read_csv("train_clean_06130930.csv")
df_test = pd.read_csv("test_clean_06130930.csv")

In [28]:
df = pd.read_csv("train_clean_06111230.csv")
df_test = pd.read_csv("test_clean_06111230.csv")

In [10]:
df = pd.read_csv("train_clean_06131000.csv")
df_test = pd.read_csv("test_clean_06131000.csv")

In [13]:
X = df.drop(["is_arrested"],axis=1)
y = df["is_arrested"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
forest = RandomForestClassifier(n_estimators=600,class_weight="balanced")
forest.fit(X_train,y_train)
y_predict_forest = forest.predict_proba(X_test)[:,1]
roc_auc_score(y_test.values,y_predict_forest)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [15]:
%%time
xgbc = xgb.XGBClassifier(max_depth=4,n_estimators=200)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.9025331559891716
cross validation score: 0.8835994083206202
CPU times: user 12min 49s, sys: 7.08 s, total: 12min 56s
Wall time: 13min 1s


In [14]:
%%time
xgbc = xgb.XGBClassifier(max_depth=4,n_estimators=100)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.8903891470401686
cross validation score: 0.8826770732964917
CPU times: user 6min 31s, sys: 6.58 s, total: 6min 38s
Wall time: 6min 37s


In [ ]:
%%time
xgbc = xgb.XGBClassifier(max_depth=5,n_estimators=100)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

<table class="table2">
    <caption>06121820</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>??? / ???</td><td>0.8990 / 0.8768</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>??? / >???</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>???? / ????</td><td>0.9181 / 0.8734</td><td>?? / ???</td><td>???,???</td></tr>
  <tr><td>700</td><td>0.9054 / 0.8726</td><td>???? / ????</td><td>??? / ???</td><td>???,???</td>
</table>


<table class="table2">
    <caption>06111230</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>0.8903 / 0.8768</td><td>??? / 0.8872?</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>0.9093 / 0.8793</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>0.9004,0.8791</td><td>0.9184 / 0.8811</td><td>0.9376 / 0.8699</td><td>???,???</td></tr>
  <tr><td>700</td><td>???,???</td><td>0.9272 / 0.8809</td><td>??? / ???</td><td>???,???</td>
</table>

In [11]:
%%time
lr = LogisticRegression(class_weight="balanced",C=0.01)
lr.fit(X_train,y_train)
y_predict_lr = lr.predict_proba(X_test)[:,1]
y_train_predict_lr = lr.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lr))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lr))

train data score: 0.8789702389514487
cross validation score: 0.8734070340927917
CPU times: user 2.48 s, sys: 659 ms, total: 3.14 s
Wall time: 2.94 s


In [14]:
%%time
mlp = MLPClassifier(alpha=1e-2,hidden_layer_sizes=6, random_state=1)
mlp.fit(X_train,y_train)
y_predict_mlp = mlp.predict_proba(X_test)[:,1]
y_train_predict_mlp = mlp.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_mlp))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_mlp))

train data score: 0.8825157026294327
cross validation score: 0.8728587660207082
CPU times: user 27.2 s, sys: 3.91 s, total: 31.1 s
Wall time: 17.4 s


<table>
  <caption>clean_06121840</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-2</td>
    <td>10</td> 
    <td>0.8877</td>
    <td>0.8893</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8887</td>
    <td>0.8825</td>
  </tr> 
</table>
<table>
  <caption>clean_06111230</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-5</td>
    <td>50</td> 
    <td>0.96</td>
    <td>0.82</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>30</td> 
    <td>0.946</td>
    <td>0.8284</td>
  </tr>    
  <tr>
    <td>1e-5</td>
    <td>20</td> 
    <td>0.9258</td>
    <td>0.8543</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>10</td> 
    <td>0.9072</td>
    <td>0.8718</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>6</td> 
    <td>0.8934</td>
    <td>0.8802</td>
  </tr>   
  <tr>
    <td>1e-3</td>
    <td>6</td> 
    <td>0.8923</td>
    <td>0.8828</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8910</td>
    <td>0.8834</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>3</td> 
    <td>0.8747</td>
    <td>0.8612</td>
  </tr>   
</table>

In [27]:
rate = np.array([0,0.,35,0])
roc_auc_score(y_test.values,np.dot(rate,np.array([y_predict_forest,y_predict_lr,y_predict_xgb,y_predict_mlp]))/np.sum(rate))

NameError: name 'y_predict_forest' is not defined

In [66]:
roc_auc_score([0,0,0,1],[0.8,0.8,0.7,0.6])*1000000000000

0.0

In [51]:
X_final_test = df_test

In [47]:
lr_final = LogisticRegression(class_weight="balanced",C=0.1)
lr_final.fit(X,y)
forest_final = RandomForestClassifier(n_estimators=600,class_weight="balanced")
forest_final.fit(X,y)


AttributeError: 'XGBClassifier' object has no attribute 'XGBClassifier'

In [49]:
xgb_final = xgb.XGBClassifier(max_depth=5,n_estimators=200)
xgb_final.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [52]:
y_final_predict_lr = lr_final.predict_proba(X_final_test)[:,1]
y_final_predict_forest = forest_final.predict_proba(X_final_test)[:,1]
y_final_predict_xgb = xgb_final.predict_proba(X_final_test)[:,1]
rate = np.array([25,0.50,35])
y_final_predict = np.dot(rate,np.array([y_final_predict_forest,y_final_predict_lr,y_final_predict_xgb]))/np.sum(rate)

In [53]:
y_final_predict = np.round(y_final_predict,6)

In [54]:
np.average(y_final_predict)

0.025188902663999833

In [55]:
submission = pd.DataFrame(data=y_final_predict, columns=['is_arrested'])
submission.head(3)

,is_arrested
0,0.006267
1,0.017358
2,0.399901


In [56]:
submission.to_csv("submission.csv",index=False)